<h1> Segmenting and Clustering Neighborhoods in Toronto</h1>

*This notebook contains the answers for all 3 parts of the peer-graded assignment.
<br>Please scroll down to view parts 2 and 3.
&nbsp;
<br>Thanks, Reha*

<h2>Part 1: Create Dataframe</h2>

<h3>Pre-cleaned Data</h3>

Import any dependencies that may be needed to scrape the wikipedia page. Quite a few were added because the notebook was experiencing parsing errors so multiple solutions were attempted.

In [57]:
import numpy as np
import pandas as pd
import requests
#!conda install -c conda-forge bs4 --yes
import bs4
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Dependencies successfully installed.')

Dependencies successfully installed.


Set the URL and soup the page.

In [8]:
wikiUrl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
getPage = requests.get(wikiUrl).text
souped = BeautifulSoup(getPage, 'html.parser')

print('Success')

Success


Search the page for the table and find all rows in the table.

In [9]:
table = souped.find('table')
tr = table.find_all('tr')

print('Success')

Success


Parse through the rows and columns of the table and create a new dataframe with the three columns assigned.

In [10]:
data = []
for row in tr:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])

print('Success')

Success


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.4+ KB


<h3>Cleaned Data to Meet Requirements</h3>

The code below refers back to the code stated in the previous section. It creates the dataframe and takes it a step further to complete the requirements of the assignemnt. In these cells we ignore table cells with a borough that is not assigned and also groups neighborhoods by postal code.

In [24]:
wikiUrl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
getPage = requests.get(wikiUrl).text
souped = BeautifulSoup(getPage, 'html.parser')

table = souped.find('table')
tr = table.find_all('tr')

data = []
for row in tr:
    data.append([t.text.strip() for t in row.find_all('td')])

dfCleaned = pd.DataFrame(data, columns=['Postalcode', 'Borough', 'Neighbourhood']) # three columns: PostalCode, Borough, and Neighborhood

dfCleaned.drop(dfCleaned[dfCleaned['Borough']=="Not assigned"].index,axis=0, inplace=True) # ignore cells with a borough that is Not assigned

dfCleaned = dfCleaned.groupby('Postalcode').agg(lambda x: ','.join(x))

dfCleaned.loc[dfCleaned['Neighbourhood']=="Not assigned",'Neighbourhood']=dfCleaned.loc[dfCleaned['Neighbourhood']=="Not assigned",'Borough']

print('Success')

Success


In [25]:
dfCleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Borough        103 non-null    object
 1   Neighbourhood  103 non-null    object
dtypes: object(2)
memory usage: 7.4+ KB


In [26]:
dfCleaned['Borough']= dfCleaned['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
dfCleaned.head(10)


,Borough,Neighbourhood
Postalcode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [27]:
dfCleaned.shape

(103, 2)

<h2>Part 2: Latitude & Longitude</h2>

In [35]:
#!conda install -c conda-forge geopy --yes
import geopy
from  geopy.geocoders import Nominatim

print('Success')

Success


The cell below reads geospatial data, renames columns to match the instructions, and then proceeds to merge the tables based on the Postal Code.

In [63]:
geoData = pd.read_csv('http://cocl.us/Geospatial_data')

geoData.rename(columns={'Postal Code':'Postalcode'},inplace=True)
dfCleaned.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)

mergedData = pd.merge(geoData, dfCleaned, on='Postalcode')
mergedData = mergedData[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
mergedData.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h2>Part 3: Explore & Cluster</h2>

First we need to initialize the credentials for the API

In [38]:
CLIENT_ID = 'RIGJPJJEEDR4UIWFSGNAYNMMTV1HKD5QI1USLLMLHXVOXGI0' # your Foursquare ID
CLIENT_SECRET = 'PP3WT3JSYJYPBK4FTFCFM3QSQ3JAKZKDFBDRREMN2L1ZSJDS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RIGJPJJEEDR4UIWFSGNAYNMMTV1HKD5QI1USLLMLHXVOXGI0
CLIENT_SECRET:PP3WT3JSYJYPBK4FTFCFM3QSQ3JAKZKDFBDRREMN2L1ZSJDS


In [49]:
neighborhood_latitude = mergedData.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = mergedData.loc[0, 'Longitude'] # neighborhood longitude value
borough_name = mergedData.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Scarborough are 43.806686299999996, -79.19435340000001.


In [62]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2ae5c1f773f578f1fe06bd'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

Initialize a map

In [61]:
map_tnt = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mergedData['Latitude'], mergedData['Longitude'], mergedData['Borough'], mergedData['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tnt)  
    
map_tnt

In [64]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighborhood_latitude, 
            neighborhood_longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
venues = getNearbyVenues(names=mergedData['Neighborhood'],
                                   latitudes=mergedData['Latitude'],
                                   longitudes=mergedData['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,1,1,1,1,1,1
"Alderwood, Long Branch",1,1,1,1,1,1
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
Bayview Village,1,1,1,1,1,1
"Bedford Park, Lawrence Manor East",1,1,1,1,1,1
...,...,...,...,...,...,...
"Willowdale, Willowdale West",1,1,1,1,1,1
Woburn,1,1,1,1,1,1
Woodbine Heights,1,1,1,1,1,1


There are 1 uniques categories.
